In [4]:
import glob
import pandas as pd
import re
import os

def create_list_of_app_ids():
    path = '../../var/data/'
    files = glob.glob(path + '/*.csv')
    files_list = []
    for filename in files:
        df = pd.read_csv(filename, index_col=None, header=0)
        files_list.append(df)
        
    data = pd.concat(files_list, axis=0, ignore_index=True)
    return(data)

In [5]:
# Description of APP's
data = create_list_of_app_ids()

In [6]:
data.head()

,Unnamed: 0,rank,title,icon,↓Total ratings,Installs,Average rating,Growth (30 days),Growth (60 days),Price,app_id,category,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion
0,0,1.0,MARVEL Future Fight,NaN,"2,815,239",50.0 M,4.29,0.3%,0.6%,Free,com.netmarble.mherosgb,GAME_ROLE_PLAYING,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2.0,Avakin Life - 3D Virtual World,NaN,"2,776,895",50.0 M,4.25,1.7%,4.0%,Free,com.lockwoodpublishing.avakinlife,GAME_ROLE_PLAYING,NaN,NaN,NaN,NaN,NaN,NaN
2,2,3.0,Shadow Fight 3,NaN,"2,755,053",50.0 M,3.93,1.1%,2.7%,Free,com.nekki.shadowfight3,GAME_ROLE_PLAYING,NaN,NaN,NaN,NaN,NaN,NaN
3,3,4.0,Angry Birds Epic RPG,NaN,"2,654,258",10.0 M,4.55,0.0%,0.0%,Free,com.rovio.gold,GAME_ROLE_PLAYING,NaN,NaN,NaN,NaN,NaN,NaN
4,4,5.0,Flight Pilot Simulator 3D Free,NaN,"2,554,187",100.0 M,4.21,0.6%,1.1%,Free,com.fungames.flightpilot,GAME_ROLE_PLAYING,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df = data[data['rank'] < 21]
list_of_ids = df['app_id']

In [8]:
# Open Comments data that were scraped as most relevant
list_of_paths = []
for app in list_of_ids:
    paths = '../../var/data/{0}'.format(app)
    list_of_paths.append(paths)
    
list_of_files = []
for path in list_of_paths:
    files = glob.glob(path + '/MOST_RELEVANT*.csv')
    list_of_files.append(files)
    
list_of_files = list(filter(None, list_of_files))
list_of_files = [y for x in list_of_files for y in x]

df_list = []
for filename in list_of_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    df['file_origin'] = filename
    df_list.append(df)

data = pd.concat(df_list, axis=0, ignore_index=True)

In [14]:
data.shape

(33700, 9)

In [15]:
list(data)

['Unnamed: 0',
 'userName',
 'userImage',
 'content',
 'score',
 'thumbsUpCount',
 'reviewCreatedVersion',
 'at',
 'file_origin']

In [16]:
data.drop_duplicates(['content', 'at']).shape

(32900, 9)

In [22]:
# Get application id
data['app_id'] = data['file_origin'].str.extract(pat='../../var/data/(.*)/MOST_RELEVANT.csv')

In [26]:
# Create list of unwanted apps
list_of_unwanted_apps = [
    'com.whatsapp',
    'com.instagram.android',
    'com.facebook.katana',
    'com.google.android.youtube',
    'com.facebook.orca',
    'com.google.android.gms',
    'com.google.android.apps.photos',
    'com.zhiliaoapp.musically',
    'com.android.chrome',
    'com.snapchat.android',
    'com.spotify.music'
]

In [30]:
# Remove apps that are not games
data = data[~data['app_id'].isin(list_of_unwanted_apps)].copy()

In [46]:
# Sample of contents
data['content'][0:4]

0    Devs não ligam pros seus jogadores. Praticas e...
1    Melhor jogo ligado a marvel. Atualiza muito rá...
2    O jogo precisa de melhorias urgentes! Acompanh...
3    A graça do jogo é vc jogar e adquirir as recom...
Name: content, dtype: object

In [47]:
data['content'][0]

'Devs não ligam pros seus jogadores. Praticas e praticas pra sugar dinheiro dos jogadores, não tem conteúdo, não somos recompensados adequadamente e por aí vai. A comunidade dá várias dicas (em todos os lugares possíveis), mas os devs não escutam quem fez esse jogo crescer, quem fez esse jogo se tornar o que se tornou, quem investe dinheiro mensalmente em conteúdo e quem se empolga com os diversos updates. (Feito por um grande fã da Marvel e que acredita que esse jogo pode ser melhor).'

In [48]:
data['content'][1000]

'Ja joguei uma vez esse jogo faz muito tempo cheguei na fase 250 mas depois parei de jogar e agora volteia jogar por que etendi qual é o lema desse jogo e agora voltei preparado para jogar merece 5 estrelas faz a versão dele so que o terceiro dele como se fosse a sequência do 2'

In [49]:
data['content'][4000]

'Não consigo vincular meu jogo com a conta do Facebook. Ele tenta vincular, mas não consegue. Instalei esse jogo pra chegar no nivel 100 e ganhar diamantes em outro jogo. A única forma de provar meu desempenho é através do Facebook. O jogo é divertido e tal, mas sem o vínculo, que garantia eu tenho que não vou perder meu progresso de uma hora para outra?'

In [50]:
data['content'][6000]

'O jogo é excelente. Permite que a gente avance muito mesmo sem investir dinheiro. Muita gente reclama, mas tendo internet e um pouco de paciência, da pra ir longe no jogo. Já estou na missão 684, e já cumpri 78 das 83 conquistas desbloqueadas, conqueistei uma medalha de ouro e uma de bronze(até agora) nos eventos especiais sem investir grana. Estou jogando apenas desde o dia 26 de março e não jogo todos os dias (por falta de tempo). Mas é um jogo de ótima jogabilidade, gráfico excelente!'

In [51]:
# Export data
data.to_csv('~/Downloads/most_relevant_comments.csv')

In [52]:
# Export sampled data
data_sampled = data.sample(n=100, random_state=123456)
data_sampled.to_csv('~/Downloads/most_relevant_comments_sampled.csv')

In [53]:
data.tail()

,Unnamed: 0,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,file_origin,app_id
27995,95,Arthur Gomes,https://lh3.googleusercontent.com/-ItGAhrnAQK4...,"Caras esse sim e um bom jogo, eu jogando esse ...",5,0,1.01,2020-05-06 13:25:55,../../var/data/com.robtopx.geometrydashmeltdow...,NaN
27996,96,igor chini,https://lh3.googleusercontent.com/a-/AOh14Gieg...,Este jogo me deixou feliz pq eu zerei então oq...,5,10,NaN,2018-09-25 00:44:45,../../var/data/com.robtopx.geometrydashmeltdow...,NaN
27997,97,BERNARDO PRÓ DE CASTRO,https://lh3.googleusercontent.com/-uRl6vCgKfs0...,Já passei de todas as frases muito legal recom...,5,1,NaN,2019-12-20 20:42:41,../../var/data/com.robtopx.geometrydashmeltdow...,NaN
27998,98,Marcio Cruz,https://lh3.googleusercontent.com/a-/AOh14GjpZ...,Musica exelente graficos simples mas fixes jog...,5,0,NaN,2020-01-16 14:51:53,../../var/data/com.robtopx.geometrydashmeltdow...,NaN
27999,99,Sinezio Mendonça,https://lh3.googleusercontent.com/-_wuqCIYjv0Q...,Acompanho esse jogo desde o inicio ja zerei to...,5,2,NaN,2020-02-25 01:25:25,../../var/data/com.robtopx.geometrydashmeltdow...,NaN


In [54]:
data['userImage'][100]

'https://lh3.googleusercontent.com/-8ucqSD86Q8g/AAAAAAAAAAI/AAAAAAAAAAA/AMZuucmujBcRWp5fTs22RSMn0WcYKtQJNg/photo.jpg'